In [15]:
from collections import namedtuple
import os
os.chdir("../")


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    train_dir: Path
    test_dir: Path

In [17]:
from ASLD_step2.constants import *
from ASLD_step2.utils import read_yaml, create_directories


CONFIG_FILE_PATH = Path("configs/config.yaml")

In [18]:
from ASLD_step2.constants import *
from ASLD_step2.utils import read_yaml,create_directories
from ASLD_step2.entity import DataIngestionConfig
from ASLD_step2.Exception import ASLDException
import sys
import shutil
class Configuration:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH
                ):
        #reading the config.yaml file by importing from constants
        self.config = read_yaml(config_filepath)
        #creating parent directories entity
        create_directories([self.config.artifacts_root])


    """
    this function is written to create the skeletion of DataIngestion entity 
    and will return path for DataIngestionConfig
    """  
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        #read config file to get data_ingestion attributes of DataIngestion
        try:
            config = self.config.data_ingestion
            #create directories for attributes of data_ingestion
            create_directories([config.root_dir])
            create_directories([config.train_dir])
            create_directories([config.test_dir])
            
            #assign the entity with its properities and return it
            data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir,
                train_dir = config.train_dir,
                test_dir = config.test_dir
            )

            return data_ingestion_config  
        except Exception as e:
            raise ASLDException(e,sys)


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil

def split_videos(self):
        # Load the CSV file
        csv_file = os.path.join(self.config.unzip_dir,'videos_id.csv')
        df = pd.read_csv(csv_file)
        captions = df['caption'].tolist()  # Get the captions column
        ids = df['ID'].tolist()  # Get the IDs column

        # Create a dictionary mapping IDs to captions
        id_to_caption = dict(zip(ids, captions))

        # List all video files in the video folder
        video_files = [f for f in os.listdir(self.config.unzip_dir) if f.endswith(".mp4")]

        # Split the videos
        train_ratio = 0.80
        videos_train, videos_test = train_test_split(video_files, train_size=train_ratio, random_state=42)

        # Create train and test folders if they don't exist
        train_folder = self.config.train_dir
        test_folder =  self.config.test_dir
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(test_folder, exist_ok=True)

        # Move videos to train folder and label them
        for video in videos_train:
            video_id = os.path.splitext(video)[0]
            caption = id_to_caption.get(video_id, "Unknown")  # Get the caption for the video ID
            labeled_video = f"{caption}_{video}"
            shutil.move(os.path.join(video_folder, video), os.path.join(train_folder, labeled_video))

        # Move videos to test folder and label them
        for video in videos_test:
            video_id = os.path.splitext(video)[0]
            caption = id_to_caption.get(video_id, "Unknown")  # Get the caption for the video ID
            labeled_video = f"{caption}_{video}"
            shutil.move(os.path.join(video_folder, video), os.path.join(test_folder, labeled_video))

        print("Videos split into train and test sets.")
